In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import torch
import random

In [2]:
purchase_df = pd.read_csv('../data/purchase.csv')
boxes_df = pd.read_csv('../data/boxes.csv')

In [3]:
purchase_df.head()

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0


In [4]:
# Use the dates to apply a day number to each purchase, offset from the base date, 1st October 2018
base_date = datetime.date(2018, 10, 1)
purchase_df['day'] = purchase_df['PURCHASE_DATE'].apply(lambda x: (datetime.datetime.strptime(x, '%d/%m/%Y').date() - base_date).days)

In [5]:
purchase_df.sample(10)

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,day
1583436,27/12/2018,290F8A32671,231.0,1.0,87
1510757,31/12/2018,297683F79B3,258.0,1.0,91
1247599,10/12/2018,2C392AD0080,99.0,1.0,70
1678193,4/11/2018,2C0F3DCABB1,143.0,1.0,34
853085,15/1/2019,2CA6BF3724C,80.0,1.0,106
978763,23/1/2019,2C24830C009,143.0,1.0,114
179789,10/2/2019,28E10C1EDD7,258.0,1.0,132
1010899,25/1/2019,2C99E0A60DC,106.0,1.0,116
354845,18/2/2019,29F80E22FF5,204.0,1.0,140
1253771,10/12/2018,2C24CC2F1D6,41.0,1.0,70


In [6]:
# We don't need the date anymore
purchase_df.drop(['PURCHASE_DATE'], axis=1, inplace=True)

In [7]:
customer_keys = purchase_df['MAGIC_KEY'].unique()
customer_ids = {customer_keys[i]: i for i in range(len(customer_keys))}

In [8]:
# Drop the invalid data
purchase_df.drop(purchase_df[purchase_df['BOX_ID'].isnull()].index, inplace=True)
purchase_df.drop(purchase_df[purchase_df['BOX_COUNT'] < 0].index, inplace=True)

In [9]:
purchase_df['BOX_COUNT'] = purchase_df['BOX_COUNT'].astype(int)
purchase_df['BOX_ID'] = purchase_df['BOX_ID'].astype(int)

In [10]:
purchase_df.head(10)

,MAGIC_KEY,BOX_ID,BOX_COUNT,day
0,2CED678A247,12,1,123
1,2BF58D91BA1,12,1,123
2,2C15B86534E,99,1,123
3,2C32D9A859A,6,1,123
4,2C7A55404D1,4,1,123
5,29D969045C2,238,1,123
6,28E5EA49074,227,1,123
7,2CEFA3A8659,6,1,123
8,2A00DE30F46,204,1,123
9,291C04B5CBF,231,1,123


In [11]:
print(len(purchase_df['MAGIC_KEY'].unique()), len(purchase_df))

1274087 2455800


In [12]:
# Add BOX_ID, BOX_COUNT, day to each of the purchases of a customer
purchases = [[] for _ in range(len(customer_keys))]
purchase_df.apply(lambda x: purchases[customer_ids[x['MAGIC_KEY']]].append((x['BOX_ID'], x['BOX_COUNT'], x['day'])), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
2455859    None
2455860    None
2455861    None
2455862    None
2455863    None
Length: 2455800, dtype: object

In [13]:
prob3_df = pd.read_csv('../data/problem 3.csv')
prob3_df.head()

,MAGIC_KEY
0,2BCFE9C06A7
1,2C2A872B5A2
2,2C6A897671B
3,2C6F1287F53
4,2C658198CC9


In [14]:
len(prob3_df)

5379

In [15]:
prob3_customers = [customer_ids[x] for x in prob3_df['MAGIC_KEY'].unique()]
len(prob3_customers)

5379

In [16]:
# Sort all the puchases of each customer by day
for i, _ in enumerate(purchases):
    purchases[i].sort(key=lambda x: x[2])

In [17]:
# Analyze single and long purchases
a = 10
b = 10
print("single purchase: ")
for i, p in enumerate(purchases):
    if len(p) == 1:
        if a > 0:
            print(i, p)
            a -= 1

print("more than 5 purchase: ")
for i, p in enumerate(purchases):
    if len(p) > 5:
        if b > 0:
            print(i, p)
            b -= 1


single purchase: 
0 [(12, 1, 123)]
1 [(12, 1, 123)]
3 [(6, 1, 123)]
4 [(4, 1, 123)]
6 [(227, 1, 123)]
7 [(6, 1, 123)]
10 [(12, 1, 123)]
11 [(231, 1, 123)]
13 [(278, 1, 123)]
16 [(231, 1, 123)]
more than 5 purchase: 
5 [(238, 1, 14), (238, 1, 36), (238, 1, 56), (238, 1, 80), (238, 1, 100), (238, 1, 123)]
14 [(203, 1, 35), (203, 1, 53), (203, 1, 63), (221, 1, 72), (203, 1, 81), (203, 1, 87), (238, 1, 97), (204, 1, 123), (204, 1, 135), (272, 1, 150)]
38 [(8, 1, 15), (8, 1, 30), (8, 1, 60), (8, 1, 82), (8, 1, 104), (8, 1, 123)]
39 [(238, 1, 20), (238, 1, 44), (268, 1, 54), (238, 1, 67), (238, 1, 76), (238, 1, 89), (238, 1, 98), (238, 1, 105), (228, 1, 123), (238, 1, 129), (238, 1, 133), (238, 1, 138), (238, 1, 142), (230, 1, 147), (238, 1, 150)]
42 [(260, 1, 8), (203, 1, 23), (221, 1, 38), (221, 1, 65), (271, 1, 84), (272, 1, 104), (221, 1, 123), (276, 1, 145)]
43 [(137, 1, 6), (137, 1, 23), (137, 1, 42), (137, 1, 62), (137, 1, 79), (174, 1, 97), (174, 1, 123)]
44 [(238, 1, 18), (238, 1, 1

In [20]:
print(boxes_df.head(10))
print(len(boxes_df))

   BOX_ID  QUALITY      DELIVERY_OPTION  MILK  MEAT  UNIT_PRICE
0       1  Premium  Home Delivery - CoD   0.0   2.7        9.96
1       2  Premium  Home Delivery - CoD   0.0   2.3       11.96
2       3  Premium  Home Delivery - CoD   0.0   2.4       11.96
3       4  Premium  Home Delivery - CoD   0.0   2.5       11.96
4       5  Premium  Home Delivery - CoD   0.0   2.6       11.96
5       6  Premium  Home Delivery - CoD   0.0   2.7       11.96
6       7  Premium  Home Delivery - CoD   0.0   2.8       11.96
7       8  Premium  Home Delivery - CoD  10.0   0.0       12.18
8       9  Premium  Home Delivery - CoD  10.5   0.0       12.78
9      10  Premium  Home Delivery - CoD   8.0   1.3       12.98
290


**meat_consumed:** Total meat consumed in a range of days<br>
**milk_consumed:** Total milk consumed in a range of days<br>
**meat_consumption_rate:** Meat consumed divided by the number of days<br>
**milk_consumption_rate:** Milk consumed divided by the number of days

In [23]:
def meat_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    meat_consumed = 0
    for id, cnt, _ in boxes:
        meat_consumed += cnt * boxes_df.at[id - 1, 'MEAT']
    return meat_consumed

def milk_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    milk_consumed = 0
    for id, cnt, _ in boxes:
        milk_consumed += cnt * boxes_df.at[id - 1, 'MILK']
    return milk_consumed


def meat_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return meat_consumed(cid, start, end_day) / (end_day - start)

def milk_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MILK'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return milk_consumed(cid, start, end_day) / (end_day - start)

In [24]:
last_day = purchase_df['day'].max()
last_date = datetime.date(2019, 2, 28)
first_day = purchase_df['day'].min()

In [25]:
print(meat_consumption_rate(0), milk_consumption_rate(0))

0.05357142857142857 0.2857142857142857


In [26]:
# Returns the info for the last meat consumption/purchase of a customer
def last_meat_consumption(cid, end_day=151):
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    try:
        bid, cnt, day = consumptions[-1]
    except:
        return None
    return [cnt * boxes_df.at[bid - 1, 'MEAT'], 
            end_day - day]
last_meat_consumption(3588, 150)

[4.4, 27]

In [27]:
split = int(len(purchases) * 0.1)
idx = [i for i in range(len(purchases))]
random.shuffle(idx)
val_idx = idx[:split]
train_idx = idx[split:]

def generate_batch(day, batch_size, split='train'):
    batch = []
    labels = []
    n = 0
    if split == 'train':
        random.shuffle(train_idx)
        current_idx = train_idx
    else:
        random.shuffle(val_idx)
        current_idx = val_idx
    for cid in current_idx:
        if len(purchases[cid]) < 2:
            continue
        last_meat = last_meat_consumption(cid, day)
        if last_meat is not None:
            rate1 = meat_consumption_rate(cid)
            rate2 = milk_consumption_rate(cid)
            next_meat = meat_consumed(cid, day, day + 15)
            if next_meat > 0.0:
                features = [rate1, rate2]
                features.extend(last_meat)
                batch.append(features)
                labels.append(next_meat)
                n += 1
                if split=='train' and n == batch_size:
                    return batch, labels
    if split == 'train':
        return None, None
    else:
        return batch, labels
batch, labels = generate_batch(15, 10)
print(batch)
print(labels)

[[0.06428571428571428, 0.35714285714285715, 1.8, 4], [0.03513513513513514, 0.0, 2.7, 12], [0.06137931034482759, 0.496551724137931, 1.3, 9], [0.060810810810810814, 0.47297297297297297, 1.8, 12], [0.08333333333333333, 0.0, 3.3, 8], [0.08243243243243242, 0.6486486486486487, 1.8, 12], [0.05496688741721855, 0.33112582781456956, 1.8, 15], [0.06266666666666666, 0.5333333333333333, 1.8, 14], [0.08933333333333332, 0.7466666666666667, 1.8, 14], [0.10596026490066225, 0.0, 3.6, 15]]
[1.8, 2.5, 1.8, 1.8, 2.9, 2.0, 1.8, 1.8, 1.8, 2.9]


In [28]:
import gc

class MeatPredictor(torch.nn.Module):
    def __init__(self):
        super(MeatPredictor, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(4, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1), 
            torch.nn.ReLU()
        )
    def forward(self, x):
        return self.mlp(x).reshape(-1)

model = MeatPredictor().cuda()
model(torch.tensor(batch, dtype=torch.float32).cuda())

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [29]:
torch.cuda.empty_cache()
gc.collect()

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
best_loss = 1e9
batch_size = 64
epochs = 100

val, val_labels = generate_batch(136, split, 'val')
val = torch.tensor(val, dtype=torch.float32).cuda()
val_labels = torch.tensor(val_labels, dtype=torch.float32).cuda()

for epoch in range(epochs):
    batch, labels = generate_batch(random.randint(70, 136), batch_size)
    if batch is None:
        continue
    batch = torch.tensor(batch, dtype=torch.float32).cuda()
    labels = torch.tensor(labels, dtype=torch.float32).cuda()
    optimizer.zero_grad()
    outputs = model(batch)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(epoch, loss.item())
    if epoch % 20 == 0:
        val_outputs = model(val)
        val_loss = loss_fn(val_outputs, val_labels)
        print("Validation loss: ", val_loss.item())
        if val_loss.item() < best_loss:
            torch.save(model.state_dict(), 'meat_predictor.pth')
            best_loss = val_loss.item()

model.load_state_dict(torch.load('meat_predictor.pth'))

0 6.657500267028809
Validation loss:  7.489947319030762
10 7.5082807540893555
20 5.893280982971191
Validation loss:  7.489947319030762
30 5.422968864440918
40 6.466249465942383
Validation loss:  7.489947319030762
50 5.910624980926514
60 6.572343826293945
Validation loss:  7.489947319030762
70 5.269999980926514
80 6.495625019073486
Validation loss:  7.489947319030762
90 5.917031288146973


<All keys matched successfully>

In [30]:
print("best loss: ", best_loss)

best loss:  7.489947319030762


In [31]:
def predict(cid, day):
    rate1 = meat_consumption_rate(cid)
    rate2 = milk_consumption_rate(cid)
    last_meat = last_meat_consumption(cid, day)
    if last_meat is not None:
        features = [rate1, rate2]
        features.extend(last_meat)
        return model(torch.tensor(features, dtype=torch.float32).cuda()).item()
    return 0.0

In [32]:

data = []
for cid in prob3_customers:
    data.append({
        'MAGIC_KEY': customer_keys[cid],
        'MEAT': predict(cid, 151)
    })
sub_df = pd.DataFrame(data, columns=['MAGIC_KEY', 'MEAT'])
sub_df.to_csv('submission.csv', index=False)